In [1]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

/var/folders/gl/l6pb7jfx42l02zc9950kh3fh0000gn/T/ipykernel_19599/632010323.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
wiki_data = pd.read_csv('wiki_languages_most_recent.csv')
# wiki_data

In [24]:
unesco_data = pd.read_csv('extinct_languages_links.csv')
# unesco_data

In [15]:
glottolog_data = pd.read_csv('languages_and_dialects_geo.csv')
glottolog_data

,glottocode,name,isocodes,level,macroarea,latitude,longitude
0,3adt1234,3Ad-Tekles,NaN,dialect,Africa,NaN,NaN
1,aala1237,Aalawa,NaN,dialect,Papunesia,NaN,NaN
2,aant1238,Aantantara,NaN,dialect,Papunesia,NaN,NaN
3,aari1239,Aari,aiw,language,Africa,5.95034,36.5721
4,aari1240,Aariya,aay,language,Eurasia,NaN,NaN
...,...,...,...,...,...,...,...
22106,zuwa1238,Zuwadza,NaN,dialect,Papunesia,NaN,NaN
22107,zwal1238,Zwall,NaN,dialect,Africa,NaN,NaN
22108,zyph1238,Zyphe,zyp,language,Eurasia,22.52400,93.2640
22109,zyud1238,Zyuzdin,NaN,dialect,Eurasia,NaN,NaN


In [26]:
# Glottolog is very specific in naming dialects and other varieties, so iso codes will likely be more navigable

wiki_lang = wiki_data['lang'].to_list()
unesco_lang = unesco_data['Name in English'].to_list()
wiki_unesco_lang = pd.DataFrame({'Agg': list(set(wiki_lang) & set(unesco_lang))})

glottolog_lang = pd.DataFrame({'Glotto lang': glottolog_data['name']})
print(len(set(set(glottolog_lang['Glotto lang']).intersection(set(wiki_unesco_lang['Agg'])))))


9


## Merging

In [28]:
geodata = pd.read_csv('languages_and_dialects_geo.csv')

langs = pd.read_csv('Extinct languages - DATA SUMMARY.csv')

In [29]:
geodata.head()

,glottocode,name,isocodes,level,macroarea,latitude,longitude
0,3adt1234,3Ad-Tekles,NaN,dialect,Africa,NaN,NaN
1,aala1237,Aalawa,NaN,dialect,Papunesia,NaN,NaN
2,aant1238,Aantantara,NaN,dialect,Papunesia,NaN,NaN
3,aari1239,Aari,aiw,language,Africa,5.95034,36.5721
4,aari1240,Aariya,aay,language,Eurasia,NaN,NaN


In [30]:
langs.head()

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered


In [36]:
len(set(geodata['name'].str.lower().values).intersection(set(langs['Name in English'].str.lower().values)))

1315

In [37]:
geodata['name_normalized'] = geodata['name'].str.lower()
langs['name_normalized'] = langs['Name in English'].str.lower()

merged = pd.merge(geodata, langs, left_on='name_normalized', right_on='name_normalized', how='inner')

In [39]:
merged['level'].value_counts()

level
language    1167
dialect      153
Name: count, dtype: int64

In [40]:
merged.to_csv('merged.csv', index=False)